In [1]:
%cd /home/angus_munro/projects/ex3/INF368-Ex3

from keras.models import load_model
from keras.callbacks import CSVLogger
from keras.optimizers import SGD, Adam

import os

from create_model import create_base_network, in_dim, tripletize, std_triplet_loss, alt_triplet_loss
from generators import triplet_generator
import testing as T

import config as C

last = C.last

/home/angus_munro/projects/ex3/INF368-Ex3


Using TensorFlow backend.


### Testing triplet_generator ###
0 a: (4, 299, 299, 3) p: (4, 299, 299, 3) n: (4, 299, 299, 3) y: (4, 2)
1 a: (4, 299, 299, 3) p: (4, 299, 299, 3) n: (4, 299, 299, 3) y: (4, 2)
2 a: (4, 299, 299, 3) p: (4, 299, 299, 3) n: (4, 299, 299, 3) y: (4, 2)
3 a: (4, 299, 299, 3) p: (4, 299, 299, 3) n: (4, 299, 299, 3) y: (4, 2)


It was found that the default base model, which was fully trainable, led to validation loss remaining maxed out at 5.0 during training, at least for 50 epochs. Training loss decreased in a reasonable way. It is likely that the weights in the Inception body were thrown off by the influence of the new last layer weights. To remedy this, weights of the new layer alone were first trained:

In [2]:
from am_plankton import set_trainable_layers

In [3]:
base_model = create_base_network(in_dim)

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
set_trainable_layers(base_model, 1)
model = tripletize(base_model)
#model.compile(optimizer=SGD(lr=C.learn_rate, momentum=0.9),
#             loss=std_triplet_loss())
model.compile(optimizer=Adam(),
             loss=alt_triplet_loss())

def avg(x):
    return sum(x)/len(x)


In [40]:
#vs = T.get_vectors(base_model, C.val_dir)

In [5]:
gen = triplet_generator(100, None ,C.val_dir)
#[a,p,n], y = next(gen)
X, y = next(gen)

In [6]:
model.evaluate(X, y, verbose=0) #display the score

4.821909637451172

In [ ]:
logger = CSVLogger(C.logfile, append=True, separator='\t')
model.fit_generator(
        triplet_generator(C.batch_size, None, C.train_dir), steps_per_epoch=300, epochs=C.iterations,
        callbacks=[logger],
        validation_data=triplet_generator(C.batch_size, None, C.val_dir), validation_steps=100)

Epoch 1/5
300/300 [==============================] - 274s 915ms/step - loss: 3.1558 - val_loss: 4.0172
Epoch 2/5
300/300 [==============================] - 241s 802ms/step - loss: 2.7693 - val_loss: 3.8837
Epoch 3/5
281/300 [===========================>..] - ETA: 12s - loss: 2.6571

In [ ]:
model.evaluate(X, y, verbose=0) #display the score

In [ ]:
set_trainable_layers(base_model, 2)
model.compile(optimizer=Adam(),
             loss=alt_triplet_loss())
model.fit_generator(
        triplet_generator(C.batch_size, None, C.train_dir), steps_per_epoch=300, epochs=C.iterations,
        callbacks=[logger],
        validation_data=triplet_generator(C.batch_size, None, C.val_dir), validation_steps=100)

Epoch 1/5
300/300 [==============================] - 220s 734ms/step - loss: 2.5259 - val_loss: 3.8521
Epoch 2/5
158/300 [==============>...............] - ETA: 1:13 - loss: 2.4744

In [28]:
len(model.layers)

5

In [29]:
model.layers

In [34]:
[l.trainable for l in model.layers]

[False, False, False, True, True]

In [47]:
sum(l.trainable for l in base_model.layers)

1

In [39]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
model_3 (Model)                 (None, 64)           21933920    input_9[0][0]                    
                                                                 input_10[0][0]                   
          

In [46]:
model